## Data preparation


### Reading Data


- read_data from dataset

- convert images to gray scale

- convert each image to a vector of 10304 value

- stack 400 vector into a single data matrix



In [8]:
import cv2
import numpy as np

path = "dataset\\s%d\\%d.pgm"

def read_data (): 

    data_matrix = np.empty((0, 10304))  
    label_vector = np.array([])
    for folder_counter in range (1, 41):
        for image_counter in range (1, 11):
            image = cv2.imread(path%(folder_counter,image_counter))
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            flattened_image = gray_image.flatten()
            flattened_image = np.array(flattened_image)
            data_matrix = np.append(data_matrix, [flattened_image], axis=0)
            label_vector = np.append(label_vector, folder_counter)
            # data_matrix = np.vstack((data_matrix, flattened_image))
    
    print(data_matrix.shape)

    return data_matrix, label_vector


dm, lv = read_data()
print(lv)

(400, 10304)
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  2.  2.  2.
  2.  2.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  8.  8.
  8.  8.  8.  8.  8.  8.  8.  8.  9.  9.  9.  9.  9.  9.  9.  9.  9.  9.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 11. 11. 11. 11. 11. 11. 11. 11.
 11. 11. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12. 13. 13. 13. 13. 13. 13.
 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 17. 17.
 17. 17. 17. 17. 17. 17. 17. 17. 18. 18. 18. 18. 18. 18. 18. 18. 18. 18.
 19. 19. 19. 19. 19. 19. 19. 19. 19. 19. 20. 20. 20. 20. 20. 20. 20. 20.
 20. 20. 21. 21. 21. 21. 21. 21. 21. 21. 21. 21. 22. 22. 22. 22. 22. 22.
 22. 22. 22. 22. 23. 23. 23. 23. 23. 23. 23. 23. 23. 23. 24. 24. 24. 24.
 24. 24. 24. 24. 24. 24. 25. 25. 25. 2

### Splitting data to training and testing data

In [13]:


def split_data(data_matrix, label_vector):
    
    test_data = data_matrix[::2] #even  
    training_data = data_matrix[1::2] #odd

    test_labels = label_vector[::2]
    training_labels = label_vector[1::2] 

    return test_data, training_data, training_labels, test_labels



## PCA Algorithm

In [3]:

def PCA(training_data, alpha):

    mean_vector = np.mean(training_data, axis=0)
   
    centered_training_data = training_data - mean_vector

    covariance_matrix = np.cov(np.transpose(centered_training_data), bias= True)
   
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
   

    sorted_indices = np.argsort(eigenvalues)[::-1]
   
    sorted_eigenvalues = eigenvalues[sorted_indices]

    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    accum = sorted_eigenvalues[0]
  
    sum = np.sum(sorted_eigenvalues)
  
    i = 1
   
    while( accum / sum < alpha):
        accum+=sorted_eigenvalues[i]
        i+=1

    projection_matrix = sorted_eigenvectors[:, :i]
  
    print("multiplying the two matrices")
    reduced_dimensionality_data = np.transpose(projection_matrix).dot(np.transpose(training_data))

    return np.transpose(reduced_dimensionality_data), projection_matrix



## PCA Classification analysis

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

data_matrix, label_vector = read_data()

test_data, training_data, training_labels, test_labels = split_data(data_matrix, label_vector)

reduced_dimensionality_data, projection_matrix = PCA(training_data, 0.95)

print("after PCA")

reduced_dimensionality_test_data = np.transpose(projection_matrix).dot(np.transpose(test_data))
reduced_dimensionality_test_data = np.transpose(reduced_dimensionality_test_data)


print("after reducing test data")

knn_classifier = KNeighborsClassifier(n_neighbors=1)

knn_classifier.fit(reduced_dimensionality_data, training_labels)

print("after fit")

test_pred = knn_classifier.predict(reduced_dimensionality_test_data)

print("after prediction")

accuracy = accuracy_score(test_labels, test_pred)

print("Accuracy:", accuracy)


(400, 10304)


NameError: name 'split_data' is not defined

## Accuracy for every value of alpha using PCA.

as we see when alpha increase the accuracy increases.

but for alpha = 0.95 the accuracy decreases due to some reasons:
- Overfitting: many components might be including noise or irrelevant details from the data that can lead to overfitting
- Curse of dimensionality: when you have many features data becomes sparse and distances between points become less meaningful
- Not all variance is useful: PCA tries to keep the components that explain the most variance, but not all variance is useful for classification.

|  Alpha   | Accuracy |
| -------- | -------- |
| 0.8      | 0.93     |
| 0.85     | 0.94     |
| 0.9      | 0.945    |
| 0.95     | 0.935    |


## LDA 

In [28]:
def LDA(training_data):
    
    
    mean_vector = np.mean(training_data, axis=0)
    sb=0
    s=0
    for i in range (0, 40):
        class_i=training_data[i*5:i*5+5]
        class_mean_i = np.mean(class_i,axis=0)
        sb+=5*np.outer(class_mean_i-mean_vector,class_mean_i-mean_vector)
        z =class_i - class_mean_i
        s+= np.transpose(z).dot(z)
        
    
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.inv(s).dot(sb))
    
    sorted_indices = np.argsort(eigenvalues)[::-1]

    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    projection_matrix = sorted_eigenvectors[:, :39]
    
    reduced_dimensionality_data = np.transpose(projection_matrix).dot(np.transpose(training_data))

    return np.transpose(reduced_dimensionality_data), projection_matrix
        
        
        
        
data_matrix, label_vector = read_data()

test_data, training_data, training_labels, test_labels = split_data(data_matrix, label_vector) 
LDA(training_data)  
        

(400, 10304)
False


In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score



data_matrix, label_vector = read_data()

test_data, training_data, training_labels, test_labels = split_data(data_matrix, label_vector)

reduced_dimensionality_data, projection_matrix = LDA(training_data)

reduced_dimensionality_test_data = np.transpose(projection_matrix).dot(np.transpose(test_data))
reduced_dimensionality_test_data = np.transpose(reduced_dimensionality_test_data)

knn_classifier = KNeighborsClassifier(n_neighbors=1)

knn_classifier.fit(reduced_dimensionality_data, training_labels)

test_pred = knn_classifier.predict(reduced_dimensionality_test_data)

accuracy = accuracy_score(test_labels, test_pred)

print("Accuracy:", accuracy)



(400, 10304)
Accuracy: 0.935


### Classifier Tuning

In [ ]:
def test_with_k(reduced_dimensionality_data,training_labels,reduced_dimensionality_test_data,test_labels,k):
    knn_classifier = KNeighborsClassifier(n_neighbors=k)

    knn_classifier.fit(reduced_dimensionality_data, training_labels)

    test_pred = knn_classifier.predict(reduced_dimensionality_test_data)

    accuracy = accuracy_score(test_labels, test_pred)

    print("Accuracy:", accuracy)
    return accuracy

#### LDA with k=1,3,5,7

In [ ]:
import matplotlib.pyplot as plt

data_matrix, label_vector = read_data()

test_data, training_data, training_labels, test_labels = split_data(data_matrix, label_vector)

reduced_dimensionality_data, projection_matrix = LDA(training_data)

reduced_dimensionality_test_data = np.transpose(projection_matrix).dot(np.transpose(test_data))
reduced_dimensionality_test_data = np.transpose(reduced_dimensionality_test_data)

LDA_performance_with_recpect_to_k = []
for i in range (1,8,2):
    LDA_performance_with_recpect_to_k.append(test_with_k(reduced_dimensionality_data,training_labels,reduced_dimensionality_test_data,test_labels,i))
    
plt.plot(list(range(1,8,2)), LDA_performance_with_recpect_to_k , marker='o', linestyle='-')

# Add labels and title
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.title('2D Data Plot')

# Show the plot
plt.grid(True)
plt.show()

#### PCA with k=1,3,5,7

In [31]:
import matplotlib.pyplot as plt

data_matrix, label_vector = read_data()

test_data, training_data, training_labels, test_labels = split_data(data_matrix, label_vector)

reduced_dimensionality_data, projection_matrix = PCA(training_data)

reduced_dimensionality_test_data = np.transpose(projection_matrix).dot(np.transpose(test_data))
reduced_dimensionality_test_data = np.transpose(reduced_dimensionality_test_data)

PCA_performance_with_recpect_to_k = []
for i in range (1,8,2):
    LDA_performance_with_recpect_to_k.append(test_with_k(reduced_dimensionality_data,training_labels,reduced_dimensionality_test_data,test_labels,i))
    
plt.plot(list(range(1,8,2)), LDA_performance_with_recpect_to_k , marker='o', linestyle='-')

# Add labels and title
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.title('2D Data Plot')

# Show the plot
plt.grid(True)
plt.show()

[[1, 2], [3, 4]]
